In [1]:
#!pip install "ray[tune]" optuna

In [2]:
import torch
import torch.optim as optim
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test
import optuna

In [3]:
get_data_loaders

<function ray.tune.examples.mnist_pytorch.get_data_loaders()>

In [4]:
def train_mnist(trial):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    train_loader, test_loader = get_data_loaders()
    print(train_loader, test_loader)
    model = ConvNet().to(device)

    optimizer = optim.SGD(
       model.parameters(),
       lr=trial.suggest_loguniform("lr", 1e-4, 1e-2),
       momentum=trial.suggest_uniform("momentum", 0.1, 0.9))

    for i in range(20):
        train(model, optimizer, train_loader, device)
        acc = test(model, test_loader, device)
        trial.report(-acc, i)
    return -acc

In [5]:
import time
start = time.time()
study = optuna.create_study()
study.optimize(train_mnist, n_jobs=1, n_trials=10)
taken = time.time() - start
print(f"Time taken: {taken:.2f} seconds.")
print(f"Best config: {study.best_params}")

[I 2021-03-04 09:51:51,350] A new study created in memory with name: no-name-041f10e5-3ed9-4fe8-a194-3deac39c8034


<torch.utils.data.dataloader.DataLoader object at 0x000001BB3E138400> <torch.utils.data.dataloader.DataLoader object at 0x000001BB3E155940>


[I 2021-03-04 09:52:00,277] Trial 0 finished with value: -0.709375 and parameters: {'lr': 0.002294876682481699, 'momentum': 0.2685036978384988}. Best is trial 0 with value: -0.709375.


<torch.utils.data.dataloader.DataLoader object at 0x000001BB3E155C18> <torch.utils.data.dataloader.DataLoader object at 0x000001BB3E155A20>


[I 2021-03-04 09:52:08,106] Trial 1 finished with value: -0.465625 and parameters: {'lr': 0.002344175318635213, 'momentum': 0.2080562577223681}. Best is trial 0 with value: -0.709375.


...

<torch.utils.data.dataloader.DataLoader object at 0x000001BB3E176908> <torch.utils.data.dataloader.DataLoader object at 0x000001BB3E176940>


[I 2021-03-04 09:53:04,597] Trial 8 finished with value: -0.875 and parameters: {'lr': 0.0041249417845283754, 'momentum': 0.6337815233330691}. Best is trial 8 with value: -0.875.


<torch.utils.data.dataloader.DataLoader object at 0x000001BB3E176C50> <torch.utils.data.dataloader.DataLoader object at 0x000001BB3E176C88>


[I 2021-03-04 09:53:12,954] Trial 9 finished with value: -0.309375 and parameters: {'lr': 0.0011924877221832489, 'momentum': 0.3926131701798178}. Best is trial 8 with value: -0.875.


Time taken: 81.61 seconds.
Best config: {'lr': 0.0041249417845283754, 'momentum': 0.6337815233330691}


In [1]:
import torch
import torch.optim as optim
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test
from ray import tune
from ray.tune.suggest.optuna import OptunaSearch


In [2]:
def train_mnist(config):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    train_loader, test_loader = get_data_loaders()
    model = ConvNet().to(device)
    optimizer = optim.SGD(
       model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(20):
        train(model, optimizer, train_loader, device)
        acc = test(model, test_loader, device)
        tune.report(mean_accuracy=acc)


In [3]:
import time
start = time.time()
analysis = tune.run(
   train_mnist,
   config={
       "lr": tune.loguniform(1e-4, 1e-2),
       "momentum": tune.uniform(0.1, 0.9),
   },
   metric="mean_accuracy",
   mode="max",
   search_alg=OptunaSearch(),
   num_samples=5)
taken = time.time() - start
print(f"Time taken: {taken:.2f} seconds.")
print(f"Best config: {analysis.best_config}")


2021-03-04 10:52:16,092	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-04 10:52:25,379	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
[I 2021-03-04 10:52:25,417] A new study created in memory with name: optuna
2021-03-04 10:52:28,371	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


Trial name,status,loc,lr,momentum
train_mnist_9c8ed192,RUNNING,,0.000707492,0.239582


2021-03-04 10:52:30,809	WARNING worker.py:1107 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 69a6825d641b461327313d1c01000000. Traceback:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\function_manager.py", line 496, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\__init__.py", line 2, in <module>
    from ray.tune.tune import run_experiments, run
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\tune.py", line 13, in <module>
    from ray.tune.ray_trial_executor import RayTrialExecutor
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 17, in <module>
    from ray.tune.durable_trainable import DurableTrainable
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\durable_trainable.py", line 5, in <module>
    from ray.tune.syncer import get_cloud_sync_client
  File "C:\ProgramData\Anacon

2021-03-04 10:52:30,858	WARNING worker.py:1107 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 1e9d04d3b7e4dfb2e14dadbf01000000. Traceback:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\function_manager.py", line 496, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\__init__.py", line 2, in <module>
    from ray.tune.tune import run_experiments, run
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\tune.py", line 13, in <module>
    from ray.tune.ray_trial_executor import RayTrialExecutor
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 17, in <module>
    from ray.tune.durable_trainable import DurableTrainable
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\durable_trainable.py", line 5, in <module>
    from ray.tune.syncer import get_cloud_sync_client
  File "C:\ProgramData\Anacon

AttributeError: 'NoneType' object has no attribute 'get'

In [4]:
import torch
import torch.optim as optim
from ray import tune
from ray.tune.examples.mnist_pytorch import get_data_loaders, ConvNet, train, test


def train_mnist(config):
    train_loader, test_loader = get_data_loaders()
    model = ConvNet(config)
    optimizer = optim.SGD(model.parameters(), lr=config["lr"])
    for i in range(30):
        train(model, optimizer, train_loader, torch.device("cpu"))
        acc = test(model, test_loader, torch.device("cpu"))
        tune.track.log(mean_accuracy=acc)

analysis = tune.run(
    train_mnist,
    config={"lr": tune.grid_search([0.001, 0.01, 0.1])})

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))

# Get a dataframe for analyzing trial results.
df = analysis.dataframe()

Trial name,status,loc,lr
train_mnist_1c2f7_00000,RUNNING,,0.001


2021-03-04 10:56:04,356	WARNING worker.py:1107 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID c6953afc4a9f69e91488ca7c01000000. Traceback:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\function_manager.py", line 496, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\__init__.py", line 2, in <module>
    from ray.tune.tune import run_experiments, run
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\tune.py", line 13, in <module>
    from ray.tune.ray_trial_executor import RayTrialExecutor
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 17, in <module>
    from ray.tune.durable_trainable import DurableTrainable
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\durable_trainable.py", line 5, in <module>
    from ray.tune.syncer import get_cloud_sync_client
  File "C:\ProgramData\Anacon

(pid=16500) Windows fatal exception: access violation
(pid=16500) 
(pid=15828) 2021-03-04 10:56:04,427	ERROR function_manager.py:498 -- Failed to load actor class ImplicitFunc.
(pid=15828) Traceback (most recent call last):
(pid=15828)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\function_manager.py", line 496, in _load_actor_class_from_gcs
(pid=15828)     actor_class = pickle.loads(pickled_class)
(pid=15828)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\__init__.py", line 2, in <module>
(pid=15828)     from ray.tune.tune import run_experiments, run
(pid=15828)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\tune.py", line 13, in <module>
(pid=15828)     from ray.tune.ray_trial_executor import RayTrialExecutor
(pid=15828)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 17, in <module>
(pid=15828)     from ray.tune.durable_trainable import DurableTrainable
(pid=15828)   File "C:\ProgramData\Anaconda3\lib\site-p

2021-03-04 10:56:04,534	ERROR trial_runner.py:616 -- Trial train_mnist_1c2f7_00002: Error processing event.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\trial_runner.py", line 586, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 609, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\_private\client_mode_hook.py", line 47, in wrapper
    return func(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\worker.py", line 1456, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(AttributeError): ray::ImplicitFunc.train_buffered() (pid=24308, ip=192.168.43.112)
  File "python\ray\_raylet.pyx", line 439, in ray._raylet.execute_task
  File "C:\ProgramData\Anaconda3\lib\site-packages\ray\worker.py", line 176,

Result for train_mnist_1c2f7_00000:
  {}
  
Result for train_mnist_1c2f7_00001:
  {}
  
Result for train_mnist_1c2f7_00002:
  {}
  


(pid=15828) Windows fatal exception: access violation
(pid=15828) 
(pid=24308) 2021-03-04 10:56:04,479	ERROR function_manager.py:498 -- Failed to load actor class ImplicitFunc.
(pid=24308) Traceback (most recent call last):
(pid=24308)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\function_manager.py", line 496, in _load_actor_class_from_gcs
(pid=24308)     actor_class = pickle.loads(pickled_class)
(pid=24308)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\__init__.py", line 2, in <module>
(pid=24308)     from ray.tune.tune import run_experiments, run
(pid=24308)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\tune.py", line 13, in <module>
(pid=24308)     from ray.tune.ray_trial_executor import RayTrialExecutor
(pid=24308)   File "C:\ProgramData\Anaconda3\lib\site-packages\ray\tune\ray_trial_executor.py", line 17, in <module>
(pid=24308)     from ray.tune.durable_trainable import DurableTrainable
(pid=24308)   File "C:\ProgramData\Anaconda3\lib\site-p

TuneError: ('Trials did not complete', [train_mnist_1c2f7_00000, train_mnist_1c2f7_00001, train_mnist_1c2f7_00002])

(pid=24308) Windows fatal exception: access violation
(pid=24308) 
(pid=32640) Windows fatal exception: access violation
(pid=32640) 
(pid=22172) Windows fatal exception: access violation
(pid=22172) 
(pid=14788) Windows fatal exception: access violation
(pid=14788) 
(pid=8924) Windows fatal exception: access violation
(pid=8924) 
(pid=33932) Windows fatal exception: access violation
(pid=33932) 
